In [1]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

2024-02-05 13:52:32.588323: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('traindata2.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,2,48,193,0,0,12,12,2,...,0,1.360000,4.480000,25.000000,4.480000,1.360000,0.407407,0.776316,1.0,1.000000
1,0,2,0,82,193,0,1,17,27,5,...,1,1.533333,1.533333,15.000000,1.533333,1.533333,0.610169,0.452381,0.5,0.500000
2,0,2,0,66,193,0,2,17,19,5,...,0,3.959391,3.045685,3.283333,3.045685,3.959391,0.545455,0.717391,1.0,1.000000
3,0,2,2,82,193,0,2,12,16,5,...,1,1.733333,2.533333,15.000000,2.533333,1.733333,0.703125,0.611940,1.0,0.888889
4,0,2,1,48,193,0,2,14,26,0,...,0,5.454545,5.454545,0.183333,5.454545,5.454545,0.000000,0.500000,1.0,1.000000


In [3]:
dfMock = pd.read_csv('mockTestData.csv')
dfMock.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,redCorner_fightTime,blueCorner_fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,1,1,12.0,12.0,2.0,4.0,0.0,0.0,35.0,...,3.02,4.55,9.833333,13.733333,3.01,3.76,0.55,0.58,0.33,0.72
1,0,1,0,17.0,19.0,5.0,4.0,0.0,0.0,33.0,...,4.38,3.37,10.600000,11.416667,3.33,2.91,0.54,0.62,0.74,0.73
2,0,1,1,12.0,16.0,5.0,5.0,0.0,1.0,37.0,...,4.34,5.20,15.583333,11.100000,5.19,2.23,0.55,0.65,0.77,0.92
3,0,1,0,13.0,15.0,6.0,8.0,0.0,0.0,33.0,...,5.68,6.59,10.766667,12.950000,4.90,6.43,0.63,0.51,0.43,0.69
4,0,1,1,17.0,13.0,0.0,6.0,0.0,0.0,29.0,...,3.07,4.34,15.000000,13.266667,5.40,3.27,0.61,0.54,0.00,0.62


In [4]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6155
6155


In [5]:
print(len(dfMock))
dfMock = dfMock.dropna(inplace=False)
print(len(dfMock))

5092
5092


In [6]:
#dropping referee
df.drop('referee', axis=1, inplace=True)

#drop billings
df.drop('billing', axis=1, inplace=True)

#drop venue
df.drop('venue', axis=1, inplace=True)

#drop title_fight
df.drop('title_fight', axis=1, inplace=True)


df.head()


,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,2,12,12,2,4,0,0,35.0,...,0,1.360000,4.480000,25.000000,4.480000,1.360000,0.407407,0.776316,1.0,1.000000
1,0,2,0,17,27,5,12,1,0,34.0,...,1,1.533333,1.533333,15.000000,1.533333,1.533333,0.610169,0.452381,0.5,0.500000
2,0,2,0,17,19,5,4,0,0,33.0,...,0,3.959391,3.045685,3.283333,3.045685,3.959391,0.545455,0.717391,1.0,1.000000
3,0,2,2,12,16,5,5,0,1,37.0,...,1,1.733333,2.533333,15.000000,2.533333,1.733333,0.703125,0.611940,1.0,0.888889
4,0,2,1,14,26,0,8,0,0,33.0,...,0,5.454545,5.454545,0.183333,5.454545,5.454545,0.000000,0.500000,1.0,1.000000


In [7]:
df.rename(columns={'fightTime': 'redCorner_fightTime'}, inplace=True)
df.insert(32, 'blueCorner_fightTime', df['redCorner_fightTime'])

dfColumns = df.columns
print(len(dfColumns))

39


In [8]:
dfMockColumns = dfMock.columns
print(len(dfMockColumns))

39


In [9]:
#one hot encode redCorner_nation
column = dfMock[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
df.drop('redCorner_nation', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = dfMock[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
df.drop('blueCorner_nation', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_103.0,blueCorner_nation_107.0,blueCorner_nation_110.0,blueCorner_nation_112.0,blueCorner_nation_113.0,blueCorner_nation_114.0,blueCorner_nation_115.0,blueCorner_nation_117.0,blueCorner_nation_118.0,blueCorner_nation_119.0
0,0,2,2,12,12,2,4,0,0,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2,0,17,27,5,12,1,0,34.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2,0,17,19,5,4,0,0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2,2,12,16,5,5,0,1,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2,1,14,26,0,8,0,0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
#one hot encode redCorner_stance
column = dfMock[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
df.drop('redCorner_stance', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = dfMock[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
df.drop('blueCorner_stance', axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_118.0,blueCorner_nation_119.0,redCorner_stance_0.0,redCorner_stance_1.0,redCorner_stance_2.0,redCorner_stance_3.0,blueCorner_stance_0.0,blueCorner_stance_1.0,blueCorner_stance_2.0,blueCorner_stance_3.0
0,0,2,2,12,12,2,4,0,0,35.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,2,0,17,27,5,12,1,0,34.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,2,0,17,19,5,4,0,0,33.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,2,2,12,16,5,5,0,1,37.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,2,1,14,26,0,8,0,0,33.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
for index, row in df.iterrows():
    if row['winner'] == 1:
        df.drop(index)

df['blueCorner'] = 1
df['winner'] = df['winner'].replace(2, 1)


df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_118.0,blueCorner_nation_119.0,redCorner_stance_0.0,redCorner_stance_1.0,redCorner_stance_2.0,redCorner_stance_3.0,blueCorner_stance_0.0,blueCorner_stance_1.0,blueCorner_stance_2.0,blueCorner_stance_3.0
0,0,1,1,12,12,2,4,0,0,35.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,1,0,17,27,5,12,1,0,34.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,1,0,17,19,5,4,0,0,33.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,1,1,12,16,5,5,0,1,37.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,1,1,14,26,0,8,0,0,33.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
#one hot encode redCorner_nation
column = dfMock[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
dfMock.drop('redCorner_nation', axis=1, inplace=True)
dfMock = pd.concat([dfMock, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = dfMock[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
dfMock.drop('blueCorner_nation', axis=1, inplace=True)
dfMock = pd.concat([dfMock, df_encoded], axis=1)

dfMock.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_103.0,blueCorner_nation_107.0,blueCorner_nation_110.0,blueCorner_nation_112.0,blueCorner_nation_113.0,blueCorner_nation_114.0,blueCorner_nation_115.0,blueCorner_nation_117.0,blueCorner_nation_118.0,blueCorner_nation_119.0
0,0,1,1,12.0,12.0,2.0,4.0,0.0,0.0,35.0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,17.0,19.0,5.0,4.0,0.0,0.0,33.0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,12.0,16.0,5.0,5.0,0.0,1.0,37.0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,13.0,15.0,6.0,8.0,0.0,0.0,33.0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,1,17.0,13.0,0.0,6.0,0.0,0.0,29.0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#one hot encode redCorner_stance
column = dfMock[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
dfMock.drop('redCorner_stance', axis=1, inplace=True)
dfMock = pd.concat([dfMock, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = dfMock[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
dfMock.drop('blueCorner_stance', axis=1, inplace=True)
dfMock = pd.concat([dfMock, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_118.0,blueCorner_nation_119.0,redCorner_stance_0.0,redCorner_stance_1.0,redCorner_stance_2.0,redCorner_stance_3.0,blueCorner_stance_0.0,blueCorner_stance_1.0,blueCorner_stance_2.0,blueCorner_stance_3.0
0,0,1,1,12,12,2,4,0,0,35.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,1,0,17,27,5,12,1,0,34.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,1,0,17,19,5,4,0,0,33.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,1,1,12,16,5,5,0,1,37.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,1,1,14,26,0,8,0,0,33.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
dfColumns = df.columns
for column in dfColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0.0
redCorner_nation_3.0
redCorner_nation_4.0
redCorner_nation_5.0
redCorner_nation_6.0
redCorner_nation_7.0
redCorner_nation_8.0
redCorner_nation_9.0
redCorner_nation_10.0
redCorner_nation_11.0
redCorner_

In [15]:
dfMockColumns = dfMock.columns
for column in dfMockColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0.0
redCorner_nation_3.0
redCorner_nation_4.0
redCorner_nation_5.0
redCorner_nation_6.0
redCorner_nation_7.0
redCorner_nation_8.0
redCorner_nation_9.0
redCorner_nation_10.0
redCorner_nation_11.0
redCorner_

In [16]:
#when one hot encoding, nations that have no representation in the train set have Nan values. This sets them to zero as they would have been zero if they were represented
df.fillna(0, inplace=True)

In [17]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6155
6155


In [18]:
#x,y split
target_column = 'winner'
y = df[target_column]
X = df.drop(target_column, axis=1)

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [19]:
#limiting this to about 2021
dfMock = dfMock.iloc[:1263]
print(len(dfMock))

1263


In [20]:
#Define the deep learning model
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

#earlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)

#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(X_train, y_train, epochs=1000, batch_size=124, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
34/40 [========================>.....] - ETA: 0s - loss: 0.7208 - accuracy: 0.5818
Epoch 1: val_accuracy improved from -inf to 0.63363, saving model to best_model_weights2.h5
40/40 [==============================] - 3s 16ms/step - loss: 0.7034 - accuracy: 0.5993 - val_loss: 0.6062 - val_accuracy: 0.6336
Epoch 2/1000
29/40 [====================>.........] - ETA: 0s - loss: 0.5432 - accuracy: 0.7369
Epoch 2: val_accuracy improved from 0.63363 to 0.75955, saving model to best_model_weights2.h5
40/40 [==============================] - 0s 5ms/step - loss: 0.5339 - accuracy: 0.7455 - val_loss: 0.4980 - val_accuracy: 0.7595
Epoch 3/1000
33/40 [=======================>......] - ETA: 0s - loss: 0.4509 - accuracy: 0.8233
Epoch 3: val_accuracy improved from 0.75955 to 0.80747, saving model to best_model_weights2.h5
40/40 [==============================] - 0s 5ms/step - loss: 0.4481 - accuracy: 0.8241 - val_loss: 0.4392 - val_accuracy: 0.8075
Epoch 4/1000
26/40 [==================>...

In [21]:
#accuracy
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_test,y_hat)

#0.8545897644191714

39/39 [==============================] - 0s 2ms/step


0.8545897644191714

In [22]:
#testing the model using career statistics

# x, y split for dfMock
target_column = 'winner'
y_mock = dfMock[target_column]
X_mock = dfMock.drop(target_column, axis=1)

#accuracy
y_hat = model.predict(X_mock)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_mock,y_hat)

#0.6492478226444972



40/40 [==============================] - 0s 3ms/step


0.6492478226444972